# Import

In [5]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all

# Parameters

In [6]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



# Preparation

In [7]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics


In [8]:
queries = {
    "pageviews": {
        "file": "queries/pageviews.hql",
        "engine": "hive"
    },
    
    "automated_pageviews": {
        "file": "queries/automated_pageviews.hql",
        "engine": "hive"
    },
    "page_previews": {
        "file": "queries/page_previews.hql",
        "engine": "hive"
    },
    "unique_devices": {
        "file": "queries/unique_devices.hql",
        "engine": "hive"
    },
    "global_south_pageviews": {
        "file": "queries/global_south_pageviews.hql",
        "engine": "hive"
    },
    
    "global_south_previews": {
        "file": "queries/global_south_previews.hql",
        "engine": "hive"
    }
    
# Removing since mobile-heavy metrics are no longer tracked/reported on in the key product metrics presentations
    #"mobile-heavy_pageviews": {
    #    "file": "queries/mobile_heavy_pageviews.hql",
    #    "engine": "hive" }, 
    #"mobile-heavy_previews": {
    #   "file": "queries/mobile_heavy_previews.hql",
    #   "engine": "hive" }
    
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running automated_pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running page_previews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running unique_devices on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_previews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


# Combining and saving metrics

In [9]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['interactions'] = new_metrics.apply(lambda x: x['previews_seen'] + x['total_pageview'], axis=1)
new_metrics['gs_interactions'] = new_metrics.apply(lambda x: x['gs_previews'] + x['gs_pageviews'], axis=1)

# Removing since mobile-heavy metrics are no longer tracked/reported on in the key product metrics presentations
# new_metrics['mh_interactions'] = new_metrics.apply(lambda x: x['mh_previews'] + x['mh_pageviews'], axis=1)


In [10]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,automated_pageviews,desktop,gs_interactions,gs_pageviews,gs_previews,interactions,mh_interactions,mh_pageviews,mh_previews,mobileweb,previews_seen,total_pageview,unique_devices
month,,,,,,,,,,,,,
2021-01-01,8.549247e+08,7.136648e+09,4.515382e+09,4.097901e+09,417481377.0,2.065884e+10,952904702.0,912845594.0,40059108.0,1.094135e+10,2.258912e+09,1.839993e+10,1.786401e+09
2021-02-01,8.173998e+08,7.036574e+09,4.460031e+09,4.066807e+09,393223979.0,1.904775e+10,839504878.0,803156054.0,36348824.0,9.677573e+09,2.057133e+09,1.699062e+10,1.836925e+09
2021-03-01,7.942069e+08,7.602720e+09,4.854990e+09,4.417236e+09,437754445.0,2.082671e+10,882315143.0,845529224.0,36785919.0,1.069313e+10,2.238340e+09,1.858837e+10,1.914841e+09
2021-04-01,8.895784e+08,6.963444e+09,4.485397e+09,4.054406e+09,430991026.0,1.972396e+10,811223383.0,774565796.0,36657587.0,1.034119e+10,2.129806e+09,1.759416e+10,1.867649e+09
2021-05-01,9.013942e+08,6.823259e+09,4.593551e+09,4.164103e+09,429448469.0,1.983804e+10,803558642.0,769075525.0,34483117.0,1.066516e+10,2.054246e+09,1.778379e+10,1.863054e+09
2021-06-01,9.837699e+08,6.110733e+09,4.260437e+09,3.863987e+09,396450484.0,1.785049e+10,780696296.0,749482516.0,31213780.0,9.703934e+09,1.773353e+09,1.607714e+10,1.745605e+09
2021-07-01,8.546527e+08,6.205220e+09,4.427486e+09,4.050288e+09,377197875.0,1.809115e+10,787963591.0,757767051.0,30196540.0,9.884966e+09,1.721155e+09,1.636999e+10,1.700404e+09
2021-08-01,1.012845e+09,6.437804e+09,4.492513e+09,4.169962e+09,322550776.0,1.819956e+10,858994724.0,831021702.0,27973022.0,1.002412e+10,1.451794e+09,1.674777e+10,1.825274e+09
2021-09-01,9.116296e+08,6.293941e+09,4.342242e+09,3.949492e+09,392749815.0,1.782378e+10,821283216.0,788543740.0,32739476.0,9.454794e+09,1.812547e+09,1.601124e+10,1.690497e+09


In [11]:
metrics.to_csv(FILENAME, sep="\t")